In [2]:
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

In [64]:
loader = PyPDFLoader("/Users/luisbarajas/Documents/AGI/small_deeplearning.pdf")
data = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=12000, chunk_overlap=0)
texts = text_splitter.split_documents(data)
print(len(texts))
# for(text) in texts:
#     print(text.page_content)


168


In [2]:
from langchain.vectorstores import  Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI   
from langchain_openai import OpenAIEmbeddings
from openai import OpenAI
import pinecone
import os
import time
from pinecone import Pinecone
from pinecone import ServerlessSpec
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
PINECONE_API_ENV = os.getenv('PINECONE_API_ENV')

pc = Pinecone(api_key="")
spec = ServerlessSpec(cloud="GCP", region="us-central1")
index_name = 'reto'

In [11]:
MODEL = "text-embedding-3-small"

client = OpenAI(
    api_key=OPENAI_API_KEY
)

print(client)

pc = Pinecone(api_key="")
spec = ServerlessSpec(cloud="GCP", region="us-central1")
index_name = 'reto'

# check if index already exists (it shouldn't if this is your first run)
if index_name not in pc.list_indexes().names():
    # if does not exist, create index
    pc.create_index(
        index_name=index_name,
        dimension=1536,  # dimensionality of text-embed-3-small
        metric='cosine',
        spec=spec
    )
    # wait for index to be initialized
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

        # connect to index
index = pc.Index(index_name)
time.sleep(1)
# view index stats
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.01,
 'namespaces': {'': {'vector_count': 1000}},
 'total_vector_count': 1000}

In [2]:
from tqdm.auto import tqdm

count = 0
for i in tqdm(range(0, len(texts))): 
    # i_end = min(i + batch_size, len(texts))
    i_end = i + 1
    # texto = texts[i].page_content
    lines_batch = texts[i].page_content.split("\n")
    ids_batch = [str(n) for n in range(i, i_end)]
    res = client.embeddings.create(input=lines_batch, model=MODEL)
    embeds = [record.embedding for record in res.data]
    meta = [{'text': line} for line in lines_batch]
    to_upsert = zip(ids_batch, embeds, meta)
    # to_upsert = [{"id": id_, "values": embed, "metadata": meta} for id_, embed, meta in zip(ids_batch, embeds, meta)]
    index.upsert(vectors=list(to_upsert))

NameError: name 'texts' is not defined

In [68]:
query = "convolutional neural network"
xq = client.embeddings.create(input=query, model=MODEL).data[0].embedding
result = index.query(vector=xq, top_k=5, include_metadata=True)

In [69]:
for match in result['matches']: 
    print(f"{match['score']:.2f}: {match['metadata']['text']}")

0.74: 5.2 Convolutional networks
0.68: convolutional layer, and its computational cost,
0.50: Xconv-2dk=5maxpoolk=3reluconv-2dk=5maxpoolk=2relureshapefully-connrelufully-connˆP(Y)
0.49: Xconv-2dk=1batchnormreluconv-2dk=3p=1batchnormreluconv-2dk=1batchnorm+reluY
0.48: by a two-hidden-layer MLP to get the final C


In [10]:
llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model_name='gpt-3.5-turbo',
    temperature=0.0,
)

text_field = "combined"

vectorstore = Pinecone(
    index, client.embebd, text_field
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

AttributeError: 'OpenAI' object has no attribute 'embed_query'